In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# load student list
Tab_studenti = pd.read_csv('students.csv', names=['student'])
# file format:
# student a
# student b
# student c

In [ ]:
# absents
# simple method to ignore absent students
# need personalization based on the input
absents = []

absents = [x.upper() for x in absents]
tmp = []
for x in Tab_studenti.studenti:
    cognome = ' '.join(x.split()[:-1])
    if cognome in absents:
        tmp.append(x)
absents = tmp
print(absents)

In [ ]:
# test parameters, generic
# default:
# all exercises have 1 point
# all exercises have the same weights
number_of_exercises = 9
weights = np.asarray([1/number_of_exercises for x in range(number_of_exercises)])
points_for_execercise = np.asarray([1 for x in range(number_of_exercises)])
mark_max=10
mark_min=2

In [ ]:
# personalization of parameters, part 1
# eg. points_for_execercise[1] = 1
points_for_execercise[0] = 3
points_for_execercise[1] = 1
points_for_execercise[2] = 2
points_for_execercise[3] = 2
points_for_execercise[4] = 2
points_for_execercise[5] = 3
points_for_execercise[6] = 2
points_for_execercise[7] = 3
points_for_execercise[8] = 1
print(points_for_execercise)

In [ ]:
# personalization of parameters, part 2
# eg. weights[1] = 1
weights[0] = 3/27
weights[1] = 1/27
weights[2] = 4/27
weights[3] = 4/27
weights[4] = 4/27
weights[5] = 2/27
weights[6] = 3/27
weights[7] = 3/27
weights[8] = 3/27
print(weights.round(2))
print('this must be equal to 1 otherwise weighs are not consistent: ', np.sum(weights))

In [ ]:
# string preparation:
# just run this script
# first line: weights
# other line: table that must be filled with points collected by student.

ll = ','.join(['0 ' for x in range(number_of_exercises)])
print ('#' + ', '.join([str(x) for x in points_for_execercise]))
print('points = [')
for x in Tab_studenti.iterrows():
    print('[' + ll + '], # ' + x[1].values[0])
print(']')

In [ ]:
# copy here the output of the above command and complete it with the proper numbers

In [ ]:
# simple error check:
np_pfe = np.asarray(points_for_execercise)
for i, x in enumerate(points):
    if (x > np_pfe).any():
        print('Error more than available points for exercise')
        print(i, x)

In [ ]:
# vote calculation:
marks = []
for x in points:
    np_x = np.asarray(x)
    sing_ex = np_x / np_pfe * weights
    ex_contrib = np.sum(sing_ex) * (mark_max - mark_min)
    mark = ex_contrib + mark_min
    marks.append(mark)
# merge data matrix
df_points = pd.DataFrame(points)
df_marks = pd.DataFrame(marks, columns=['Mark'])
df_marks = pd.concat([Tab_studenti, df_points, df_marks], axis=1)

In [ ]:
# delete absent students
df_marks = df_marks[Tab_studenti['student'].isin(absents) == False]

In [ ]:
# results:
df_marks.round(2)

In [ ]:
# sufficient (italian std):
df_voti[df_voti['Voto']>= 5.75].round(2).sort_values('Voto', ascending=False)

In [ ]:
# not sufficient (italian std)
df_voti[(df_voti['Voto']<5.75) & (df_voti['Voto']>=5)].round(2).sort_values('Voto', ascending=False)

In [ ]:
# very bed marks (italian std)
df_voti[(df_voti['Voto']<5)].round(2).sort_values('Voto', ascending=False)

In [ ]:
# comments on the tests
obs="""
Comments here, latex std
"""

In [ ]:
#dump on file
import os
import subprocess

if not os.path.isdir('pdf'):
    os.mkdir('pdf')

In [ ]:
f = open('pdf/end.tex', 'w', encoding="utf-8")
string=\
"""
\documentclass{letter}
\pagenumbering{gobble}
\\usepackage{geometry}
\\usepackage{booktabs}
\\usepackage[italian]{babel}
\\usepackage[utf8]{inputenc}
\\usepackage{amsmath}
\geometry{
 a4paper,
 right=10mm,
 left=10mm,
 top=10mm,
 bottom=10mm,
 }
\\begin{document}
"""
f.write(string)
f.write(df_voti.round(2).sort_values('Voto', ascending=False).to_latex())
f.write('\\\\\n')
f.write('\n\\bigskip\n')
f.write('\n')
f.write(pd.DataFrame([points_for_execercise, weights], index=['Punti per esercizio','Peso esercizio']).round(3).to_latex())
f.write('\\\\\n')
f.write('\n\\bigskip\n')
f.write('\n')
f.write('Assenti:\\\\\n')
f.write(', '.join(absents))
f.write('\\\\\n')
f.write('\n\\bigskip\n')
f.write('\n')
f.write('$v_{{min}}$ = {}\\\\\n'.format(voto_min))
f.write('$v_{{max}}$ = {}\\\\\n'.format(voto_max))
equation=\
"""
\\begin{equation*}
	v = v_{min} + \\sum_{i \\in ex} p_i \\frac{\\text{score}_{ex}}{\\text{score}_{tot}} (v_{max}- v_{min})
\\end{equation*}
"""
f.write(equation)
f.write('')
f.write('\\\\\n')
f.write(obs)
f.write('\\\\\n')
f.write('\n\end{document}')
f.flush()

In [ ]:
# if latex is available, compilation of the output.
subprocess.check_call(['pdflatex',
                           '-output-directory pdf',
                           './pdf/end.tex'])